In [1]:
import math
from simtk import unit
import os
import tempfile
import pickle
import mdtraj as md
import numpy as np
from simtk.unit.quantity import Quantity
import logging 

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [2]:
def get_trajs_for_state(i, aa, phase, length, out_dir, htf, index):
    new_top = md.Topology.from_openmm(htf._topology_proposal.new_topology)
    old_top = md.Topology.from_openmm(htf._topology_proposal.old_topology)
    
    # From Hannah: https://github.com/hannahbrucemacdonald/endstate_pdbs/blob/master/scripts/input_for_pol_calc.py
    from perses.analysis.utils import open_netcdf
    nc = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{aa}_{length}.nc"))
    nc_checkpoint = open_netcdf(os.path.join(out_dir, f"{i}_{phase}_{aa}_{length}_checkpoint.nc"))
    checkpoint_interval = nc_checkpoint.CheckpointInterval
    all_positions = nc_checkpoint.variables['positions']
    n_iter, n_replicas, n_atoms, _ = np.shape(all_positions)
    
    from tqdm import tqdm
    all_pos_new = np.zeros(shape=(n_iter, new_top.n_atoms, 3))
    all_pos_old = np.zeros(shape=(n_iter, old_top.n_atoms, 3))
    for iteration in tqdm(range(n_iter)):
        replica_id = np.where(nc.variables['states'][iteration*checkpoint_interval] == index)[0]
        pos = all_positions[iteration,replica_id,:,:][0] *unit.nanometers
        all_pos_new[iteration] = htf.new_positions(pos).value_in_unit_system(unit.md_unit_system) # Get new positions only
        all_pos_old[iteration] = htf.old_positions(pos).value_in_unit_system(unit.md_unit_system)

    return all_pos_new, all_pos_old

    


In [27]:
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/10/38/"
i = os.path.basename(os.path.dirname(out_dir))
phase = 'vacuum'
endstate = 0
aa = 'SER'.lower()
length = '5ns'
index = 0
with open(os.path.join(out_dir, f"{i}_{phase}_{endstate}.pickle"), "rb") as f:
    htf = pickle.load(f)


In [26]:
new, old = get_trajs_for_state(i, aa, phase, length, out_dir, htf, index)

100%|██████████| 5001/5001 [03:04<00:00, 27.07it/s]


In [24]:
# traj_new = md.Trajectory(new, md.Topology.from_openmm(htf._topology_proposal.new_topology))
# traj_new.save(out_dir + f"new_{index}.dcd")

In [23]:
# traj_new[0].save(out_dir + f"new_{index}.pdb")

In [28]:
traj_old = md.Trajectory(old, md.Topology.from_openmm(htf._topology_proposal.old_topology))
traj_old.save(out_dir + f"old_{index}.dcd")

In [29]:
traj_old[0].save(out_dir + f"old_{index}.pdb")